### Assignment : Week 1 
## Modeling simple RL problems by making their MDPs in Python

We will create the MDPs for some of the example problems from Grokking textbook. For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

Later on, you will not need to implement the MDPs of common RL problems yourself, most of the work is already done by the OpenAI Gym library, which includes models for most of the famous RL envis.

You can start this assignment during/after reading Grokking Ch-2.

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [1]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Slippery Walk

Let us now look at the BSW environment on Page 40. We'll model a slightly modified version of BSW with 7 states instead (i.e the SWF envi on Page 67). It will be useful in the coming weeks.

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [2]:
swf_mdp = {}

# Terminal states
swf_mdp[0] = {
    "Right": [(1, 0, 0, True)],
    "Left": [(1, 0, 0, True)],
}
swf_mdp[6] = {
    "Right": [(1, 6, 0, True)],
    "Left": [(1, 6, 0, True)],
}

# Non-terminal states
for state in range(1, 6):
    swf_mdp[state] = {
        "Right": [
            (1/2, min(state + 1, 6), 1 if state == 5 else 0, state + 1 == 6),
            (1/3, state, 0, False),
            (1/6, max(state - 1, 0), 0, state - 1 == 0),
        ],
        "Left": [
            (1/2, max(state - 1, 0), 1 if state == 1 else 0, state - 1 == 0),
            (1/3, state, 0, False),
            (1/6, min(state + 1, 6), 0, state + 1 == 6),
        ],
    }

import pprint
pprint.pprint(swf_mdp)


{0: {'Left': [(1, 0, 0, True)], 'Right': [(1, 0, 0, True)]},
 1: {'Left': [(0.5, 0, 1, True),
              (0.3333333333333333, 1, 0, False),
              (0.16666666666666666, 2, 0, False)],
     'Right': [(0.5, 2, 0, False),
               (0.3333333333333333, 1, 0, False),
               (0.16666666666666666, 0, 0, True)]},
 2: {'Left': [(0.5, 1, 0, False),
              (0.3333333333333333, 2, 0, False),
              (0.16666666666666666, 3, 0, False)],
     'Right': [(0.5, 3, 0, False),
               (0.3333333333333333, 2, 0, False),
               (0.16666666666666666, 1, 0, False)]},
 3: {'Left': [(0.5, 2, 0, False),
              (0.3333333333333333, 3, 0, False),
              (0.16666666666666666, 4, 0, False)],
     'Right': [(0.5, 4, 0, False),
               (0.3333333333333333, 3, 0, False),
               (0.16666666666666666, 2, 0, False)]},
 4: {'Left': [(0.5, 3, 0, False),
              (0.3333333333333333, 4, 0, False),
              (0.16666666666666666, 5, 0, 

Feel free to automate filling this MDP, but ensure that it is correctly filled as it'll be back in next week's assignment.

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [3]:
fl_mdp = {}
grid_size = 4
terminal_states = {5, 7, 11, 12, 15}

def get_next_state(state, action):
    row, col = divmod(state, grid_size)
    if action == "Up":
        row = max(0, row - 1)
    elif action == "Down":
        row = min(grid_size - 1, row + 1)
    elif action == "Left":
        col = max(0, col - 1)
    elif action == "Right":
        col = min(grid_size - 1, col + 1)
    return row * grid_size + col

for state in range(16):
    transitions = {}
    for action in ["Up", "Down", "Left", "Right"]:
        if state in terminal_states:
            transitions[action] = [(1, state, 0, True)]
        else:
            next_state = get_next_state(state, action)
            transitions[action] = [
                (0.8, next_state, 1 if next_state in terminal_states else 0, next_state in terminal_states),
                (0.1, state, 0, False),
                (0.1, get_next_state(state, "Left" if action == "Right" else "Right"), 0, False),
            ]
    fl_mdp[state] = transitions

import pprint
pprint.pprint(fl_mdp)


{0: {'Down': [(0.8, 4, 0, False), (0.1, 0, 0, False), (0.1, 1, 0, False)],
     'Left': [(0.8, 0, 0, False), (0.1, 0, 0, False), (0.1, 1, 0, False)],
     'Right': [(0.8, 1, 0, False), (0.1, 0, 0, False), (0.1, 0, 0, False)],
     'Up': [(0.8, 0, 0, False), (0.1, 0, 0, False), (0.1, 1, 0, False)]},
 1: {'Down': [(0.8, 5, 1, True), (0.1, 1, 0, False), (0.1, 2, 0, False)],
     'Left': [(0.8, 0, 0, False), (0.1, 1, 0, False), (0.1, 2, 0, False)],
     'Right': [(0.8, 2, 0, False), (0.1, 1, 0, False), (0.1, 0, 0, False)],
     'Up': [(0.8, 1, 0, False), (0.1, 1, 0, False), (0.1, 2, 0, False)]},
 2: {'Down': [(0.8, 6, 0, False), (0.1, 2, 0, False), (0.1, 3, 0, False)],
     'Left': [(0.8, 1, 0, False), (0.1, 2, 0, False), (0.1, 3, 0, False)],
     'Right': [(0.8, 3, 0, False), (0.1, 2, 0, False), (0.1, 1, 0, False)],
     'Up': [(0.8, 2, 0, False), (0.1, 2, 0, False), (0.1, 3, 0, False)]},
 3: {'Down': [(0.8, 7, 1, True), (0.1, 3, 0, False), (0.1, 3, 0, False)],
     'Left': [(0.8, 2, 0, F

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.

You don't need to worry about Gym right now, we'll set it up in the coming weeks. But here is the code to import an MDP.

In [1]:
import gym
P = gym.make('FrozenLake-v1').env.P

Since the imported MDP is also just a dictionary, we can just print it.

In [2]:
# using the pretty print module

import pprint
pprint.pprint(P)

{0: {0: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     2: [(0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)],
     3: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False)],
     2: [(0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     3: [(0.3333333333333333,